**THIS FILE IS OUT OF DATE**.
It is left there only for reference. The new PSA format for sharing data makes this notebook useless.  

# Loading socio-economic data

#### Loading functions

First we import the [pandas](http://pandas.pydata.org/) function librairy. Pandas is a standard python librairy that alows us to manipulate Excel-like tables (called DataFrames) with named rows and columns.

In [1]:
import pandas as pd

#### Reading data

Now we read the excel data into a pandas DataFrame.
We start from an Excel file that contains socio-economic data. In the future this file may for instance be populated by PSA.

In [2]:
data_from_excel= pd.read_excel("inputs/input_data_Feb2016.xlsx", #the name of the file
                        sheetname="Consolidated (2012)", #the Excel tab were the data is
                               index_col="province",#column to use as index
                               header=1, #skips the first line of the excel file
                                );
data_from_excel.index = data_from_excel.index.str.title() #fixes the case of province names in the Excel file
data_from_excel.head() #shows the first few lines of the table

,Region,Region PSGC,Province PSGC,GRDPC 2012 (At Current Prices),Projected Population 2012,"Average Annual Family Income, 2009","Average Annual Family Income, by Region, 2012",% Wages and salaries 2012,% Entrepreneurial activities 2012,% Other sources of income 2012,...,% Others Deposits 2012,% Health Expenditure 2012,% of Births by Attended Skilled Health Personnel 2012,% hh with radio 2012,% hh with landlines 2012,% hh with cellular phones 2012,"Public Schools, Elementary, 2012-2013","Public Schools, Secondary, 2012-2013",Estimated QRF 2012,Estimated LDRRM Fund 2012
province,,,,,,,,,,,,,,,,,,,,,
Abra,CAR,14,1401,126843,240135.244121,133688,257000,0.343701,0.247626,0.297152,...,0.002270,0.031414,0.857720,0.652174,0.062112,0.953416,277,33,31746830.14320,1.058228e+08
Agusan Del Norte,CARAGA,16,1602,48954,661728.454375,179014,180000,0.387500,0.224497,0.298354,...,0.000045,0.034563,0.921445,0.395745,0.027660,0.821277,293,86,40128811.09725,1.337627e+08
Agusan Del Sur,CARAGA,16,1603,48954,677779.682154,126492,180000,0.387500,0.224497,0.298354,...,0.000552,0.034563,0.727442,0.395745,0.027660,0.821277,483,95,50795871.21195,1.693196e+08
Aklan,6,6,604,57801,554414.442422,119962,202000,0.371111,0.195986,0.374721,...,0.000133,0.044318,0.806176,0.548898,0.069559,0.823003,320,70,34597652.21625,1.153255e+08
Albay,5,5,505,38870,1264097.894966,158629,162000,0.384548,0.211663,0.335946,...,0.003677,0.032568,0.840840,0.514019,0.024299,0.800000,601,122,61822427.32725,2.060748e+08


This table contains more data (more columns) that what we need to run the model. In addition, the names of the coumn are human-readable, instead of correspondig to variable names in the model. Finally, Some data is missing. We solve each one of this problems in the following.

### Matching columns in the Excel file to variables in the model

#### pov_head, pop,shew

Some of the data in the Excel file match directly data in the model. We can transform them directly using a simple dictionary, [inputs/data_source_matching.csv](inputs/data_source_matching.csv), that matches the name in the Excel file to the name in the model

In [ ]:
#reads the CSV file that matches names in excel ot names in the model
data_source_matching =pd.read_csv("inputs/data_source_matching.csv",
                                  index_col="name_in_data",
                                 )
data_source_matching #displays the result

In [ ]:
#keeps only the colomns listed in data_source_matching
df=data_from_excel[data_source_matching.index]
#renames those columns to their name in the model
df=df.rename(columns=data_source_matching["name_in_model"])
df.head()

Some other model variables do not match directly one column in the data.


#### Provincincial GDP extrapolated from regional gdp growth

We have income at the provincial level in 2009 and at the regional level only in 2012. Here we approximate regional income in 2009 averaging provincial income in 2009 (weighted by population in 2012), then we assume all provinces grew homogenously at their regional growth rate between 2009 and 2012.

(We do this for demonstration purposes, since provincial income for a recent date is probably available otherwise)

To compute the weighted average, we use another standard python library, [NumPy](http://www.numpy.org/) the provides  standard mathematical functions such as log, exp, weighted average, etc.

In [ ]:
import numpy as np

In [ ]:
#renames a few useful columns for convinience
h=data_from_excel[["Region", "Projected Population 2012", "Average Annual Family Income, 2009",
                   "Average Annual  Family Income, by Region, 2012"
                  ]].rename(columns={"Projected Population 2012":"pop",
                                     "Average Annual Family Income, 2009": "gp9",
                                     "Average Annual  Family Income, by Region, 2012":"gr12"
                                    }).dropna()

# in each region, multiplies provincial income in 2009 by average regional growth between 2009 and 2012
df["gdp_pc_pp"]=h.groupby("Region").apply(lambda x: x["gp9"]*x["gr12"]/(np.average(x["gp9"], weights=x["pop"]))
                                         ).reset_index("Region",drop=True) / 5  #5 is the average family size

#Changes the unit of GDP to thousands of pesos (technical: to reduce risk of float overflows when computing welfare)
df["gdp_pc_pp"]/=1e3


The model needs income information in each province to be provided relative to the average income in the Philippines.
Witin each province, we need the income of the poor and nonpoor households relative to the average income in the province.


In [ ]:
#National average income 
df["gdp_pc_pp_nat"] = np.average(df.dropna().gdp_pc_pp,  weights=df.dropna()["pop"]) #note that we have to manually remove the lines with missing data (.dropna()) because numpy does not handle missing data

#Average income of poor households (estimated from Rio as avg income of the poorest quitile)
wp=80.5/5

df["cp"] = wp
df["cr"]= (df["gdp_pc_pp"] - df["pov_head"]*df["cp"])/(1-df["pov_head"]) 

df.head()

#### Access to savings, transfers

In [ ]:
#share of income from transfers: we use the same value for poor and nonpoor, and we sum two columns of the input data
df["social_p"]=df["social_r"]=data_from_excel[["% Other sources of income 2012","% Other receipts 2012"]].sum(axis=1)

df.head()

# Loading data on exposure, hazard, and protection

### Exposure (population in flood-prone areas)

Exposure comes from different files, for instance they could be provided by DOST.

#### River floods

In [ ]:
#Exposure to RIVER floods (at present from GLOFRIS)
pop_exposed = pd.read_csv("inputs/pop_exposed.csv",index_col=["NAME_1"])
pop_exposed.index=pop_exposed.index.str.title()
pop_exposed.head()

Note how for some provinces (Aklan, Albay) are not exposed to river floods according to our data source. Also, the data we have here is for several return periods. The model can work either with on single return period or several return periods. The information on different exposed periods sorted in a different variable, `fa_ratios`.

First we define the exposure (Fraction of people Affected) as the one corresponding to 10 yr return period.

In [ ]:
expo_river = pop_exposed[["rp10_pop"]]
expo_river.columns=["fa"]
expo_river.head()

Then we define the exposure to other return period events relative to the exposure to the 10yr event

In [ ]:
fa_ratios_river = pop_exposed.div(expo_river.squeeze(),axis=0)
fa_ratios_river.columns=[10,100]
fa_ratios_river.columns.name="rp"
fa_ratios_river.index.name="province"
fa_ratios_river.head()

#### Coastal floods

Now coastal floods

In [ ]:
pd.read_csv("inputs/exposure_to_coastal_foods.csv").head() #inspecting the raw data

In [ ]:
expo_coast = pd.read_csv("inputs/exposure_to_coastal_foods.csv", usecols=["fa_coast", "id_1"],index_col="id_1", )
expo_coast.index = expo_coast.index.str.title() #matches case
expo_coast.columns=["fa"]
expo_coast.head()

For coastal flood the data we have has only one return period.

In [ ]:
fa_ratios_coast = pd.DataFrame(1,index=expo_coast.index, columns=[10])
fa_ratios_coast.index.name = "province"
fa_ratios_coast.columns.name="rp"
fa_ratios_coast.head(3)

#### Combining hazard and differentiating poor and nonpoor

Finally we combine the data on table with province, hzard, return period. We store multi-hazard data on a different dataframe.


In [ ]:
df_multihazard = pd.concat( [expo_coast,expo_river],keys=["coast", "river"],names=["hazard", "param"],axis=1)
df_multihazard.index.name = "province"
df_multihazard=df_multihazard.stack("hazard")
df_multihazard.head()

The model uses exposure for poor and nonpoor people separatedly.
Based on international cas estudies, poor people are about 15% more exposed to floods than nonpoor people. 

In [ ]:
dfm = pd.merge(
    df_multihazard.reset_index(), df["pov_head"].reset_index(), on="province")

pe=15/100
fa= dfm["fa"]
ph = dfm["pov_head"]

fap = fa*(1+pe)
far= (fa-ph*fap)/(1-ph)

dfm["fap"]=fap
dfm["far"]=far

dfm = dfm.set_index(["province","hazard"]).drop(["fa","pov_head"],axis=1)
# dfm.replace(0,np.nan,inplace=True) #treat no exposure as no data
# dfm=dfm.dropna(how="all")
dfm.head()

We store the the fa_ratios separatedly

In [ ]:
fa_ratios = pd.concat([fa_ratios_river, fa_ratios_coast], keys=["river","coast"], names=["hazard"], axis=1).stack("hazard").fillna(method="pad",axis=1)
fa_ratios.head()

In [ ]:
fa_ratios.to_csv("inputs/fa_ratios.csv")

#### Files for single-hazard version of the model

In [ ]:
fa_ratios_river.to_csv("inputs/fa_ratios_river.csv") #this one stored to have an example of single hzard multiple return periods.

In [ ]:
# df["fa"] = expo_river["fa"] #just so that we can run the model with single hazard
dfmriver = dfm.query("hazard=='river'").reset_index("hazard")[["fap","far"]]
df["fap"]=dfmriver["fap"]
df["far"]=dfmriver["far"]

df.head()

### Vulnerability

To assess asset vulnerability in each province, we use census data on roof and wall types in each province.
We match these types to a given vulnerability with reduced vulnerability curves. Let us first open the files that matche wall and roof types to vulnerability.

#### Reduced vulnerability curves for wall and roofs

In [ ]:
#matches roof and wall types to vulnerabilities
roof_types_to_vuln =pd.read_csv("inputs/roof_types_to_vuln.csv").squeeze().sort_values(ascending=False)
wall_types_to_vuln =pd.read_csv("inputs/wall_types_to_vuln.csv").squeeze().sort_values(ascending=False)

print("Reduced vulnerability curve for roofs\n")
print(roof_types_to_vuln)
#print("\nReduced vulnerability curve for walls")
#print(wall_types_to_vuln)

#### Sorting roofs according to income

The data for **roof** types in each province come from the excel file with socio-economic data we used at the begining.

In [ ]:
share =data_from_excel[roof_types_to_vuln.index]
share.head()

Then we assume that the poorest households in  each province use the houses with lowest quality roofs.

In [ ]:
#sorts roof types according to income
p=(share.cumsum(axis=1).add(-df["pov_head"],axis=0)).clip(lower=0)
poor=(share-p).clip(lower=0)
rich=share-poor

print("Type of roofs for nonpoor households:")
rich.head()

Finally we average vulnerability accross roof types

In [ ]:
#averages vulnerability accross roof type
vp_roof=((poor*roof_types_to_vuln).sum(axis=1)/df["pov_head"] )
vr_roof=(rich*roof_types_to_vuln).sum(axis=1)/(1-df["pov_head"])

vp_roof.head()

#### Sorting walls according to income

Then we do the same for <b>walls</b>...

In [ ]:
#sorts wall types according to income
share =data_from_excel[wall_types_to_vuln.keys()]
p=(share.cumsum(axis=1).add(-df["pov_head"],axis=0)).clip(lower=0)
poor=(share-p).clip(lower=0)
rich=share-poor

#walls
vp_wall=((poor*wall_types_to_vuln).sum(axis=1)/df["pov_head"] )
vr_wall=(rich*wall_types_to_vuln).sum(axis=1)/(1-df["pov_head"])


...and take the average value for roof and walls.

In [ ]:
#averages value for roofs and walls
vp = (vp_roof+vp_wall)/2
vr = (vr_roof+vr_wall)/2

df["v_p"]=vp
df["v_r"]=vr

#plots
# import matplotlib.pyplot as plt
# %matplotlib inline
# vp.hist(), plt.xlabel("vp")
# vr.hist(color="red",alpha=0.5),plt.xlabel("vr")


### Hazard (protection)

We capture hazard through the protection level, given in return period. Here we use data from FLOPROS as a placeholder.
FLOPROS uses a different spelling for some province, so we correct that here.

In [ ]:
protection = pd.read_csv("inputs/protection_phl.csv",index_col="province", squeeze=True).sort_index()
protection.index = protection.index.str.title()
protection.rename(index={"Cotabato":"North Cotabato",
                         'Mindoro Occidental':"Occidental Mindoro",
                         'Mindoro Oriental':"Oriental Mindoro",}, inplace=True) #(an altenrative way would be to use and demonstrate the function replace_with_warning)
protection.head()

In [ ]:
df["protection"]=protection.clip_lower(2) #assumes at least 2yr protection
df["protection"].head()

# Manually filling data gaps and informing parameters

Some data is missing and has to be added manually

In [ ]:
#average productivity of capital
df["avg_prod_k"] = .23

#Reconstruction time (an only be guessed ex-ante)
df["T_rebuild_K"] = 3

# how much early warning reduces vulnerability (eg reactivity to early warnings)
df["pi"] = 0.2

Some other inputs are normative or policy choices

In [ ]:
#assumption on cross-provincial risk sharing
df["nat_buyout"] = 0.3

#scale up of transfers after a disaster hits
df["sigma_r"]=df["sigma_p"]=0

#income elasticity
df["income_elast"] = 1.5

#discount rate
df["rho"]=5/100

# Adds description to the variables names

Here we add a human readable descritpion to all model variables, based on the descriptions gathered in [inputs/inputs_info.csv](inputs/inputs_info.csv)

In [ ]:
description = pd.read_csv("inputs/inputs_info.csv", index_col="key")["descriptor"]
description.head()

In [ ]:
df.ix["description"]= description
data=df.T.reset_index().set_index(["description","index"]).T
data.columns.names = ['description', 'variable']
data.head().T #displays the first few provinces, transposed for ease of reading.

# Saves the data

In [ ]:
#saves the data
data.to_excel("inputs/all_data_compiled.xlsx")

**That's it, we have built an excel file with all our data!**
To see how to use this data with the resilience model, go to [socio_economic_capacity_demo.ipynb](socio_economic_capacity_demo.ipynb)



 

## Multi-hazard data

In [ ]:
dfm.to_csv("inputs/multi_hazard_data.csv")

# Report missing data by province

This code builds a table reporting missing data points for each province

In [ ]:
def write_missing_data(s):
    which = s[s.isnull()].index.values
    return ", ".join(which)

def count_missing_data(s):
    return s.isnull().sum()

report = pd.DataFrame()

report["nb_missing"]=df.apply(count_missing_data,axis=1)  
report["missing_data"]=df.apply(write_missing_data,axis=1)

report  = report.ix[report["nb_missing"]>0,:]
report.sort_values(by="nb_missing",inplace=True)
report.to_csv("inputs/missing_data_report.csv")

report

We see that for a few provinces, we have no data on protection. Let us inspect the data on protection.

In [ ]:
protection.ix[report.index]

In our data on protection, these provinces have a missing value (nan). This probelm should be investigated going back to the source used for protection (here, FLOPROS as a placeholder, but that could be relaced by a domestic source, for instance DOST)
